In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

from aif360.datasets import StandardDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric
from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.preprocessing.reweighing import Reweighing
from aif360.algorithms.inprocessing import PrejudiceRemover
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing

from sklearn.metrics import accuracy_score
from scipy.stats import mode
import pandas as pd
import numpy as np
import errno
import math
import copy
from copy import deepcopy
import csv
import tensorflow as tf
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [2]:
def preprocess_compasdataset(df):
    df = df[['age', 'c_charge_degree', 'race', 'age_cat', 'score_text',
                 'sex', 'priors_count', 'days_b_screening_arrest', 'decile_score',
                 'is_recid', 'two_year_recid', 'c_jail_in', 'c_jail_out']]

    # Indices of data samples to keep
    ix = df['days_b_screening_arrest'] <= 30
    ix = (df['days_b_screening_arrest'] >= -30) & ix
    ix = (df['is_recid'] != -1) & ix
    ix = (df['c_charge_degree'] != "O") & ix
    ix = (df['score_text'] != 'N/A') & ix
    df = df.loc[ix,:]
    df['length_of_stay'] = (pd.to_datetime(df['c_jail_out']) - pd.to_datetime(df['c_jail_in'])).apply(lambda x: x.days)

    # Restrict races to African-American and Caucasian
    df = df.loc[~df['race'].isin(['Native American','Hispanic','Asian','Other']),:]

    df = df[['sex','race','age_cat','c_charge_degree','score_text','priors_count','is_recid', 'two_year_recid','length_of_stay']]

    df['priors_count'] = df['priors_count'].apply(lambda x: 0 if x <= 0 else ('1 to 3' if 1 <= x <= 3 else 'More than 3'))
    df['length_of_stay'] = df['length_of_stay'].apply(lambda x: '<week' if x <= 7 else ('<3months' if 8 < x <= 93 else '>3months'))
    df['score_text'] = df['score_text'].apply(lambda x: 'MediumHigh' if (x == 'High')| (x == 'Medium') else x)
    df['age_cat'] = df['age_cat'].apply(lambda x: '25 to 45' if x == '25 - 45' else x)

    df['sex'] = df['sex'].replace({'Female': 1.0, 'Male': 0.0})
    df['race'] = df['race'].apply(lambda x: 1.0 if x == 'Caucasian' else 0.0)

    df = df[['two_year_recid', 'sex', 'race', 'age_cat', 'priors_count', 'c_charge_degree']]

    protected_attributes = ['sex', 'race']
    label_name = 'two_year_recid'
    categorical_features = ['age_cat', 'priors_count', 'c_charge_degree']
    features = categorical_features + [label_name] + protected_attributes

    # privileged classes
    privileged_classes = {"sex": [1.0], "race": [1.0]}

    # protected attribute maps
    protected_attribute_map = {"sex": {0.0: 'Male', 1.0: 'Female'},
                                "race": {1.0: 'Caucasian', 0.0: 'Not Caucasian'}}


    data = StandardDataset(df, label_name, favorable_classes=[0],
                           protected_attribute_names=protected_attributes,
                           privileged_classes=[privileged_classes[x] for x in protected_attributes],
                           categorical_features=categorical_features,
                           features_to_keep=features,
                           metadata={'label_maps': [{1.0: 'Did recid.', 0.0: 'No recid.'}],
                                     'protected_attribute_maps': [protected_attribute_map[x] for x in protected_attributes]})

    return data

In [3]:
def preprocess_germandataset(df):
    def group_credit_hist(x):
        if x in ['no credits taken/ all credits paid back duly', 'all credits at this bank paid back duly', 'existing credits paid back duly till now']:
            return 'None/Paid'
        elif x == 'delay in paying off in the past':
            return 'Delay'
        elif x == 'critical account/ other credits existing (not at this bank)':
            return 'Other'
        else:
            return 'NA'

    def group_employ(x):
        if x == 'unemployed':
            return 'Unemployed'
        elif x in ['... < 1 year ', '1 <= ... < 4 years']:
            return '1-4 years'
        elif x in ['4 <= ... < 7 years', '.. >= 7 years']:
            return '4+ years'
        else:
            return 'NA'

    def group_savings(x):
        if x in ['... < 100 DM', '100 <= ... < 500 DM']:
            return '<500'
        elif x in ['500 <= ... < 1000 DM ', '.. >= 1000 DM ']:
            return '500+'
        elif x == 'unknown/ no savings account':
            return 'Unknown/None'
        else:
            return 'NA'

    def group_status(x):
        if x in ['< 0 DM', '0 <= ... < 200 DM']:
            return '<200'
        elif x in ['>= 200 DM / salary assignments for at least 1 year']:
            return '200+'
        elif x == 'no checking account':
            return 'None'
        else:
            return 'NA'

    status_map = {'male : divorced/separated': 1.0,
                'male : single': 1.0,
                'male : married/widowed': 1.0,
                'female : divorced/separated/married': 0.0,
                'female : single': 0.0}

    df['personal_status_sex'] = df['personal_status_sex'].replace(status_map)
    df['credit_history'] = df['credit_history'].apply(lambda x: group_credit_hist(x))
    df['savings'] = df['savings'].apply(lambda x: group_savings(x))
    df['present_emp_since'] = df['present_emp_since'].apply(lambda x: group_employ(x))
    df['age'] = df['age'].apply(lambda x: np.float(x >= 25))
    df['account_check_status'] = df['account_check_status'].apply(lambda x: group_status(x))

    df = df.rename(columns = {'default': 'credit', 'present_emp_since': 'employment', 'account_check_status': 'status', 'personal_status_sex': 'sex'})

    protected_attribute = ['sex', 'age']
    label_name = 'credit'
    categorical_features = ['credit_history', 'savings', 'employment']
    features = categorical_features + [label_name] + protected_attribute

    privileged_class = {'sex': [1.0], 'age': [1.0]}

    protected_attribute_map = {"sex": {1.0: 'male', 0.0: 'female'},
                            "age": {1.0: 'old', 0.0: 'young'}}

    data = StandardDataset(df, label_name, favorable_classes=[1],
                            protected_attribute_names=protected_attribute,
                            privileged_classes=[privileged_class[x] for x in protected_attribute],
                            categorical_features=categorical_features,
                            features_to_keep=features,
                            metadata={'label_maps': [{1.0: 'Good Credit', 2.0: 'Bad Credit'}],
                                    'protected_attribute_maps': [protected_attribute_map[x] for x in protected_attribute]})

    return data

In [4]:
############ Reweighing ##############

def reweighing_data(train, unprivileged_group, privileged_group):
    RW = Reweighing(unprivileged_groups=unprivileged_group, privileged_groups=privileged_group)
    RW.fit(train)
    train_transformed = RW.transform(train)

    # change weights to whole numbers
    for i in range(train_transformed.instance_weights.size):
        train_transformed.instance_weights[i] = (round(train_transformed.instance_weights[i] / 0.1) * 0.1) * 10
        weights = copy.deepcopy(train_transformed.instance_weights)

    # change train_transformed.features and train_transformed.labels and train_transformed.protected_attributes according to the weights of each instance
    for i in range(train_transformed.features.shape[0]):
        row = copy.deepcopy(train_transformed.features[i])
        row_label = copy.deepcopy(train_transformed.labels[i])
        row_protected_attributes = copy.deepcopy(train_transformed.protected_attributes[i])
        row_protected_attributes.resize(1,2)
        row.resize(1,train_transformed.features.shape[1])
        row_label.resize(1,1)
        weight = int(weights[i])
        for j in range(weight-1):
            train_transformed.features = np.concatenate((train_transformed.features,row))
            train_transformed.labels = np.concatenate((train_transformed.labels,row_label))
            train_transformed.protected_attributes = np.concatenate((train_transformed.protected_attributes,row_protected_attributes))

    # change the train_transformed to a numpy array of ones to match number of rows in features
    train_transformed.instance_weights = np.ones(train_transformed.features.shape[0])

    return train_transformed

In [5]:
def output_rates(input_data, output_data, attribute_name, privileged=None, unprivileged=None, favourable=None, unfavourable=None):
    
    index_attribute = input_data.feature_names.index(attribute_name)
    privileged = float(privileged)
    unprivileged = float(unprivileged)
    
    input_priv = input_data.labels[np.where(input_data.features[:,index_attribute] == privileged)]
    input_priv = np.reshape(input_priv, (input_priv.shape[0],1))
    output_priv = output_data.labels[np.where(output_data.features[:,index_attribute] == privileged)]
    output_priv = np.reshape(output_priv, (output_priv.shape[0],1))
    
    priv_labels = np.concatenate((input_priv, output_priv), axis=1)
    
    input_unpriv = input_data.labels[np.where(input_data.features[:,index_attribute] == unprivileged)]
    input_unpriv = np.reshape(input_unpriv, (input_unpriv.shape[0],1))
    output_unpriv = output_data.labels[np.where(output_data.features[:,index_attribute] == unprivileged)]
    output_unpriv = np.reshape(output_unpriv, (output_unpriv.shape[0],1))
    
    unpriv_labels = np.concatenate((input_unpriv, output_unpriv), axis=1)
    
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    
    for i in range(priv_labels.shape[0]):
        input_label = priv_labels[i][0]
        output_label = priv_labels[i][1]
        if input_label == output_label:
            if input_label == unfavourable:
                tn = tn + 1
            else:
                tp = tp + 1
        else:
            if input_label == favourable and output_label == unfavourable:
                fn = fn + 1
            else:
                fp = fp + 1
    
    rates_privileged = [tp,fp,tn,fn]
    
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    
    for i in range(unpriv_labels.shape[0]):
        input_label = unpriv_labels[i][0]
        output_label = unpriv_labels[i][1]
        if input_label == output_label:
            if input_label == unfavourable:
                tn = tn + 1
            else:
                tp = tp + 1
        else:
            if input_label == favourable and output_label == unfavourable:
                fn = fn + 1
            else:
                fp = fp + 1
                
    rates_unprivileged = [tp,fp,tn,fn]  
    
    rates_list = [rates_privileged, rates_unprivileged]
    
    return rates_list

In [6]:
def equal_opp_diff(input_data, output_data, attribute_name, privileged, unprivileged, favourable, unfavourable):
    rates_both = output_rates(input_data, output_data, attribute_name, privileged, unprivileged, favourable, unfavourable)
    
    # [tp, fp, tn, fn]
    outcome_privileged = rates_both[0]
    outcome_unprivileged = rates_both[1]
    
    # true positive rate = tp / (tp + fn)
    tpr_privileged = outcome_privileged[0] / (outcome_privileged[0] + outcome_privileged[3])
    tpr_unprivileged = outcome_unprivileged[0] / (outcome_unprivileged[0] + outcome_unprivileged[3])

    fairness = tpr_unprivileged - tpr_privileged
    
    return fairness

In [7]:
def avg_odds_diff(input_data, output_data, attribute_name, privileged, unprivileged, favourable, unfavourable):
    rates_both = output_rates(input_data, output_data, attribute_name, privileged, unprivileged, favourable, unfavourable)
    
    # [tp, fp, tn, fn]
    outcome_privileged = rates_both[0]
    outcome_unprivileged = rates_both[1]
    
    # true positive rate = tp / (tp + fn)
    tpr_privileged = outcome_privileged[0] / (outcome_privileged[0] + outcome_privileged[3])
    tpr_unprivileged = outcome_unprivileged[0] / (outcome_unprivileged[0] + outcome_unprivileged[3])

    # false positive rate = fp / (fp + tn)
    fpr_privileged = outcome_privileged[1] / (outcome_privileged[1] + outcome_privileged[2])
    fpr_unprivileged = outcome_unprivileged[1] / (outcome_unprivileged[1] + outcome_unprivileged[2])
    
    fpr_diff = fpr_unprivileged - fpr_privileged
    tpr_diff = tpr_unprivileged - tpr_unprivileged
    
    fairness = (fpr_diff + tpr_diff) * 0.5
    
    return fairness

In [8]:
def make_prediction(train, test, unprivileged_groups, privileged_groups):
    
    ################## adversarial debiasing #################

    sess = tf.Session()
    debiased_model_reweighing = AdversarialDebiasing(privileged_groups = privileged_groups,
                                                     unprivileged_groups = unprivileged_groups,
                                                     scope_name='debiased_classifier', debias=True, sess=sess)
    debiased_model_reweighing.fit(train)
    dataset_debiasing_test_reweighing = debiased_model_reweighing.predict(test)
    sess.close()
    tf.reset_default_graph()

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_debiasing_test_reweighing,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test, dataset_debiasing_test_reweighing,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)

    accuracy_adversarial = accuracy_score(y_true = test.labels, y_pred = dataset_debiasing_test_reweighing.labels)
    
    equal_opportunity_difference = equal_opp_diff(test, dataset_debiasing_test_reweighing,
                                                  'sex', privileged=1, unprivileged=0, favourable=1, unfavourable=0)
    
    average_odds_difference = avg_odds_diff(test, dataset_debiasing_test_reweighing,
                                                  'sex', privileged=1, unprivileged=0, favourable=1, unfavourable=0)
    
    metrics_adversarial = [metric_test.mean_difference(),acc_test.disparate_impact(), equal_opportunity_difference, average_odds_difference, acc_test.theil_index()]


    ##################### prejudice remover #####################
    prejudice_model_reweighing = PrejudiceRemover(eta=100, sensitive_attr='sex')
    prejudice_model_reweighing.fit(train)
    dataset_prejudice_test_reweighing = prejudice_model_reweighing.predict(test)

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_prejudice_test_reweighing,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test, dataset_prejudice_test_reweighing,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    accuracy_prejudice = accuracy_score(y_true=test.labels, y_pred=dataset_prejudice_test_reweighing.labels)
    
    equal_opportunity_difference = equal_opp_diff(test, dataset_prejudice_test_reweighing,
                                                  'sex', privileged=1, unprivileged=0, favourable=1, unfavourable=0)
    
    average_odds_difference = avg_odds_diff(test, dataset_prejudice_test_reweighing,
                                                  'sex', privileged=1, unprivileged=0, favourable=1, unfavourable=0)
    
    if acc_test.disparate_impact() == math.inf:
        disparate_impact = 5.0
    else:
        disparate_impact = acc_test.disparate_impact()
    
    metrics_prejudice = [metric_test.mean_difference(), disparate_impact, equal_opportunity_difference, average_odds_difference, acc_test.theil_index()]


    ##################### normal neural net #####################
    sess = tf.Session()
    neural_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                                        unprivileged_groups = unprivileged_groups,
                                        scope_name='debiased_classifier', debias=False, sess=sess)
    neural_model.fit(train)
    dataset_neural_test = neural_model.predict(test)
    sess.close()
    tf.reset_default_graph()

    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_neural_test,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test, dataset_neural_test,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    accuracy_nondebiasing = accuracy_score(y_true=test.labels, y_pred=dataset_neural_test.labels)
    
    equal_opportunity_difference = equal_opp_diff(test, dataset_neural_test,
                                                  'sex', privileged=1, unprivileged=0, favourable=1, unfavourable=0)
    
    average_odds_difference = avg_odds_diff(test, dataset_neural_test,
                                                  'sex', privileged=1, unprivileged=0, favourable=1, unfavourable=0)

    metrics_nondebiasing = [metric_test.mean_difference(),acc_test.disparate_impact(), equal_opportunity_difference, average_odds_difference, acc_test.theil_index()]


    ##################### ensemble #####################
    pred_labels_test = []
    for i in range(0, len(test.features)):
        arr_test = mode([dataset_debiasing_test_reweighing.labels[i], dataset_prejudice_test_reweighing.labels[i], dataset_neural_test.labels[i]])
        pred_labels_test.append(arr_test[0][0])
        dataset_ensemble_test = test.copy()
        dataset_ensemble_test.labels = np.array(pred_labels_test)


    ##################### metrics #####################

    metric_test = BinaryLabelDatasetMetric(dataset_ensemble_test,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
    acc_test = ClassificationMetric(test, dataset_ensemble_test,
                                    unprivileged_groups=unprivileged_groups,
                                    privileged_groups=privileged_groups)
    accuracy_ensemble = accuracy_score(y_true=test.labels, y_pred=dataset_ensemble_test.labels)
    
    equal_opportunity_difference = equal_opp_diff(test, dataset_ensemble_test,
                                                  'sex', privileged=1, unprivileged=0, favourable=1, unfavourable=0)
    
    average_odds_difference = avg_odds_diff(test, dataset_ensemble_test,
                                                  'sex', privileged=1, unprivileged=0, favourable=1, unfavourable=0)
    
    metrics_ensemble = [metric_test.mean_difference(),acc_test.disparate_impact(), equal_opportunity_difference, average_odds_difference, acc_test.theil_index()]

    accuracy_scores = [accuracy_adversarial, accuracy_prejudice, accuracy_nondebiasing, accuracy_ensemble]
    fairness_metrics = [metrics_adversarial, metrics_prejudice, metrics_nondebiasing, metrics_ensemble]
    
    return accuracy_scores, fairness_metrics

In [11]:
privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]

print("Classification with Compas data set\n")
# df = pd.read_csv('dataset/compas-scores-two-years.csv')
dataset_orig = load_preproc_data_compas()

matrix_accuracy_reweigh = {}
matrix_accuracy_nonreweigh = {}
matrix_fairness_reweigh = {}
matrix_fairness_nonreweigh = {}

runs = 10

for i in range(0, runs):
    print('run =', i+1)
    
    train, test = dataset_orig.split([0.7], shuffle=True)
    train_transformed = reweighing_data(train, unprivileged_groups, privileged_groups)
    
    # with reweighing
    accuracy_reweigh, fairness_metrics_reweigh = make_prediction(train_transformed, test, unprivileged_groups, privileged_groups)
    
    # Without reweighing
    accuracy_nonreweigh, fairness_metrics_nonreweigh = make_prediction(train, test, unprivileged_groups, privileged_groups)
    
    # store values for each run
    matrix_accuracy_reweigh[i] = accuracy_reweigh
    matrix_fairness_reweigh[i] = fairness_metrics_reweigh
    matrix_accuracy_nonreweigh[i] = accuracy_nonreweigh
    matrix_fairness_nonreweigh[i] = fairness_metrics_nonreweigh

print('\nprediction completed')

metrics_adversarial_reweigh = []
metrics_prejudice_reweigh = []
metrics_nondebiasing_reweigh = []
metrics_ensemble_reweigh = []
metrics_adversarial_nonreweigh = []
metrics_prejudice_nonreweigh = []
metrics_nondebiasing_nonreweigh = []
metrics_ensemble_nonreweigh = []

for i in range(0, runs):
    
    # with reweighing
    metrics_adversarial_reweigh.append(matrix_fairness_reweigh[i][0])
    metrics_prejudice_reweigh.append(matrix_fairness_reweigh[i][1])
    metrics_nondebiasing_reweigh.append(matrix_fairness_reweigh[i][2])
    metrics_ensemble_reweigh.append(matrix_fairness_reweigh[i][3])
    
    # without reweighing
    metrics_adversarial_nonreweigh.append(matrix_fairness_nonreweigh[i][0])
    metrics_prejudice_nonreweigh.append(matrix_fairness_nonreweigh[i][1])
    metrics_nondebiasing_nonreweigh.append(matrix_fairness_nonreweigh[i][2])
    metrics_ensemble_nonreweigh.append(matrix_fairness_nonreweigh[i][3])

print('compiled all metrics')


# create data frame for all metrics
columns = ['Adversarial Debiasing', 'Prejudice Remover', 'Nondebiasing', 'Ensemble']
accuracy_reweigh = np.array(list(matrix_accuracy_reweigh.values()))
accuracy_nonreweigh = np.array(list(matrix_accuracy_nonreweigh.values()))
compas_accuracy_reweigh = pd.DataFrame(accuracy_reweigh, columns=columns)
compas_accuracy_nonreweigh = pd.DataFrame(accuracy_nonreweigh, columns=columns)

# fairness metrics
columns = ['Mean Difference', 'Disparate Impact', 'Equal Opportunity Difference', 'Average Odds Difference', 'Theil Index']
compas_adversarial_reweigh = pd.DataFrame(metrics_adversarial_reweigh, columns=columns)
compas_prejudice_reweigh = pd.DataFrame(metrics_prejudice_reweigh, columns=columns)
compas_nondebiasing_reweigh = pd.DataFrame(metrics_nondebiasing_reweigh, columns=columns)
compas_ensemble_reweigh = pd.DataFrame(metrics_ensemble_reweigh, columns=columns)

compas_adversarial_nonreweigh = pd.DataFrame(metrics_adversarial_nonreweigh, columns=columns)
compas_prejudice_nonreweigh = pd.DataFrame(metrics_prejudice_nonreweigh, columns=columns)
compas_nondebiasing_nonreweigh = pd.DataFrame(metrics_nondebiasing_nonreweigh, columns=columns)
compas_ensemble_nonreweigh = pd.DataFrame(metrics_ensemble_nonreweigh, columns=columns)


# save to csv
compas_accuracy_reweigh.to_csv("results-11feb/race/compas/reweighed/accuracy.csv", encoding='utf-8')
compas_adversarial_reweigh.to_csv("results-11feb/race/compas/reweighed/adversarial.csv", encoding='utf-8')
compas_prejudice_reweigh.to_csv("results-11feb/race/compas/reweighed/prejudice.csv", encoding='utf-8')
compas_nondebiasing_reweigh.to_csv("results-11feb/race/compas/reweighed/neural_net.csv", encoding='utf-8')
compas_ensemble_reweigh.to_csv("results-11feb/race/compas/reweighed/ensemble.csv", encoding='utf-8')

compas_accuracy_nonreweigh.to_csv("results-11feb/race/compas/non-reweighed/accuracy.csv", encoding='utf-8')
compas_adversarial_nonreweigh.to_csv("results-11feb/race/compas/non-reweighed/adversarial.csv", encoding='utf-8')
compas_prejudice_nonreweigh.to_csv("results-11feb/race/compas/non-reweighed/prejudice.csv", encoding='utf-8')
compas_nondebiasing_nonreweigh.to_csv("results-11feb/race/compas/non-reweighed/neural_net.csv", encoding='utf-8')
compas_ensemble_nonreweigh.to_csv("results-11feb/race/compas/non-reweighed/ensemble.csv", encoding='utf-8')

print('all files saved to csv')

Classification with Compas data set

run = 1
epoch 0; iter: 0; batch classifier loss: 0.695470; batch adversarial loss: 0.682815
epoch 0; iter: 200; batch classifier loss: 0.642505; batch adversarial loss: 0.685670
epoch 1; iter: 0; batch classifier loss: 0.586604; batch adversarial loss: 0.705760
epoch 1; iter: 200; batch classifier loss: 0.642258; batch adversarial loss: 0.656937
epoch 2; iter: 0; batch classifier loss: 0.679931; batch adversarial loss: 0.677651
epoch 2; iter: 200; batch classifier loss: 0.666182; batch adversarial loss: 0.687480
epoch 3; iter: 0; batch classifier loss: 0.616984; batch adversarial loss: 0.669878
epoch 3; iter: 200; batch classifier loss: 0.635190; batch adversarial loss: 0.660383
epoch 4; iter: 0; batch classifier loss: 0.614649; batch adversarial loss: 0.675361
epoch 4; iter: 200; batch classifier loss: 0.638734; batch adversarial loss: 0.624785
epoch 5; iter: 0; batch classifier loss: 0.607590; batch adversarial loss: 0.679574
epoch 5; iter: 200; b

epoch 47; iter: 200; batch classifier loss: 0.641248; batch adversarial loss: 0.655261
epoch 48; iter: 0; batch classifier loss: 0.653534; batch adversarial loss: 0.655720
epoch 48; iter: 200; batch classifier loss: 0.601478; batch adversarial loss: 0.644198
epoch 49; iter: 0; batch classifier loss: 0.588578; batch adversarial loss: 0.681721
epoch 49; iter: 200; batch classifier loss: 0.598818; batch adversarial loss: 0.685440
epoch 0; iter: 0; batch classifier loss: 0.698325
epoch 0; iter: 200; batch classifier loss: 0.603732
epoch 1; iter: 0; batch classifier loss: 0.610171
epoch 1; iter: 200; batch classifier loss: 0.621058
epoch 2; iter: 0; batch classifier loss: 0.639811
epoch 2; iter: 200; batch classifier loss: 0.621119
epoch 3; iter: 0; batch classifier loss: 0.598129
epoch 3; iter: 200; batch classifier loss: 0.605107
epoch 4; iter: 0; batch classifier loss: 0.654245
epoch 4; iter: 200; batch classifier loss: 0.626804
epoch 5; iter: 0; batch classifier loss: 0.564539
epoch 5; 

epoch 33; iter: 0; batch classifier loss: 0.590957; batch adversarial loss: 0.679993
epoch 34; iter: 0; batch classifier loss: 0.598771; batch adversarial loss: 0.663141
epoch 35; iter: 0; batch classifier loss: 0.558820; batch adversarial loss: 0.631812
epoch 36; iter: 0; batch classifier loss: 0.619083; batch adversarial loss: 0.656718
epoch 37; iter: 0; batch classifier loss: 0.648645; batch adversarial loss: 0.670215
epoch 38; iter: 0; batch classifier loss: 0.617649; batch adversarial loss: 0.652082
epoch 39; iter: 0; batch classifier loss: 0.639729; batch adversarial loss: 0.708107
epoch 40; iter: 0; batch classifier loss: 0.578672; batch adversarial loss: 0.633830
epoch 41; iter: 0; batch classifier loss: 0.586027; batch adversarial loss: 0.667069
epoch 42; iter: 0; batch classifier loss: 0.570580; batch adversarial loss: 0.649699
epoch 43; iter: 0; batch classifier loss: 0.598360; batch adversarial loss: 0.641830
epoch 44; iter: 0; batch classifier loss: 0.669714; batch adversa

epoch 25; iter: 0; batch classifier loss: 0.629506; batch adversarial loss: 0.675567
epoch 25; iter: 200; batch classifier loss: 0.589801; batch adversarial loss: 0.669937
epoch 26; iter: 0; batch classifier loss: 0.657331; batch adversarial loss: 0.701242
epoch 26; iter: 200; batch classifier loss: 0.594253; batch adversarial loss: 0.687537
epoch 27; iter: 0; batch classifier loss: 0.635627; batch adversarial loss: 0.667326
epoch 27; iter: 200; batch classifier loss: 0.599011; batch adversarial loss: 0.700772
epoch 28; iter: 0; batch classifier loss: 0.641175; batch adversarial loss: 0.673098
epoch 28; iter: 200; batch classifier loss: 0.598031; batch adversarial loss: 0.658722
epoch 29; iter: 0; batch classifier loss: 0.605377; batch adversarial loss: 0.678661
epoch 29; iter: 200; batch classifier loss: 0.580204; batch adversarial loss: 0.687107
epoch 30; iter: 0; batch classifier loss: 0.665078; batch adversarial loss: 0.675284
epoch 30; iter: 200; batch classifier loss: 0.643316; b

epoch 38; iter: 0; batch classifier loss: 0.592296
epoch 38; iter: 200; batch classifier loss: 0.631842
epoch 39; iter: 0; batch classifier loss: 0.614906
epoch 39; iter: 200; batch classifier loss: 0.593236
epoch 40; iter: 0; batch classifier loss: 0.621893
epoch 40; iter: 200; batch classifier loss: 0.618157
epoch 41; iter: 0; batch classifier loss: 0.593308
epoch 41; iter: 200; batch classifier loss: 0.649730
epoch 42; iter: 0; batch classifier loss: 0.620105
epoch 42; iter: 200; batch classifier loss: 0.586929
epoch 43; iter: 0; batch classifier loss: 0.570927
epoch 43; iter: 200; batch classifier loss: 0.661625
epoch 44; iter: 0; batch classifier loss: 0.654550
epoch 44; iter: 200; batch classifier loss: 0.663090
epoch 45; iter: 0; batch classifier loss: 0.598821
epoch 45; iter: 200; batch classifier loss: 0.625604
epoch 46; iter: 0; batch classifier loss: 0.597847
epoch 46; iter: 200; batch classifier loss: 0.630554
epoch 47; iter: 0; batch classifier loss: 0.577587
epoch 47; ite

...

epoch 4; iter: 0; batch classifier loss: 0.631836; batch adversarial loss: 0.667754
epoch 4; iter: 200; batch classifier loss: 0.674842; batch adversarial loss: 0.693835
epoch 5; iter: 0; batch classifier loss: 0.657547; batch adversarial loss: 0.671234
epoch 5; iter: 200; batch classifier loss: 0.667484; batch adversarial loss: 0.655314
epoch 6; iter: 0; batch classifier loss: 0.645303; batch adversarial loss: 0.689906
epoch 6; iter: 200; batch classifier loss: 0.618467; batch adversarial loss: 0.678802
epoch 7; iter: 0; batch classifier loss: 0.696188; batch adversarial loss: 0.696612
epoch 7; iter: 200; batch classifier loss: 0.597943; batch adversarial loss: 0.658388
epoch 8; iter: 0; batch classifier loss: 0.710157; batch adversarial loss: 0.651732
epoch 8; iter: 200; batch classifier loss: 0.585198; batch adversarial loss: 0.671327
epoch 9; iter: 0; batch classifier loss: 0.674529; batch adversarial loss: 0.646420
epoch 9; iter: 200; batch classifier loss: 0.598226; batch adversa

epoch 3; iter: 0; batch classifier loss: 0.605763
epoch 3; iter: 200; batch classifier loss: 0.611673
epoch 4; iter: 0; batch classifier loss: 0.606327
epoch 4; iter: 200; batch classifier loss: 0.665338
epoch 5; iter: 0; batch classifier loss: 0.643140
epoch 5; iter: 200; batch classifier loss: 0.665184
epoch 6; iter: 0; batch classifier loss: 0.597487
epoch 6; iter: 200; batch classifier loss: 0.652947
epoch 7; iter: 0; batch classifier loss: 0.676465
epoch 7; iter: 200; batch classifier loss: 0.578657
epoch 8; iter: 0; batch classifier loss: 0.611475
epoch 8; iter: 200; batch classifier loss: 0.698146
epoch 9; iter: 0; batch classifier loss: 0.586708
epoch 9; iter: 200; batch classifier loss: 0.610808
epoch 10; iter: 0; batch classifier loss: 0.581947
epoch 10; iter: 200; batch classifier loss: 0.636937
epoch 11; iter: 0; batch classifier loss: 0.638622
epoch 11; iter: 200; batch classifier loss: 0.628798
epoch 12; iter: 0; batch classifier loss: 0.643044
epoch 12; iter: 200; batch 

epoch 42; iter: 0; batch classifier loss: 0.616077; batch adversarial loss: 0.665513
epoch 43; iter: 0; batch classifier loss: 0.602944; batch adversarial loss: 0.649330
epoch 44; iter: 0; batch classifier loss: 0.649589; batch adversarial loss: 0.643099
epoch 45; iter: 0; batch classifier loss: 0.646933; batch adversarial loss: 0.660259
epoch 46; iter: 0; batch classifier loss: 0.634951; batch adversarial loss: 0.687801
epoch 47; iter: 0; batch classifier loss: 0.621876; batch adversarial loss: 0.637174
epoch 48; iter: 0; batch classifier loss: 0.676269; batch adversarial loss: 0.694898
epoch 49; iter: 0; batch classifier loss: 0.638579; batch adversarial loss: 0.664932
epoch 0; iter: 0; batch classifier loss: 0.694214
epoch 1; iter: 0; batch classifier loss: 0.657353
epoch 2; iter: 0; batch classifier loss: 0.597048
epoch 3; iter: 0; batch classifier loss: 0.606600
epoch 4; iter: 0; batch classifier loss: 0.644997
epoch 5; iter: 0; batch classifier loss: 0.601240
epoch 6; iter: 0; ba

epoch 29; iter: 0; batch classifier loss: 0.599842; batch adversarial loss: 0.674838
epoch 29; iter: 200; batch classifier loss: 0.592847; batch adversarial loss: 0.642261
epoch 30; iter: 0; batch classifier loss: 0.564119; batch adversarial loss: 0.679965
epoch 30; iter: 200; batch classifier loss: 0.570962; batch adversarial loss: 0.673437
epoch 31; iter: 0; batch classifier loss: 0.566300; batch adversarial loss: 0.690406
epoch 31; iter: 200; batch classifier loss: 0.578543; batch adversarial loss: 0.685791
epoch 32; iter: 0; batch classifier loss: 0.568404; batch adversarial loss: 0.685653
epoch 32; iter: 200; batch classifier loss: 0.591136; batch adversarial loss: 0.656958
epoch 33; iter: 0; batch classifier loss: 0.605338; batch adversarial loss: 0.685496
epoch 33; iter: 200; batch classifier loss: 0.637445; batch adversarial loss: 0.700784
epoch 34; iter: 0; batch classifier loss: 0.556603; batch adversarial loss: 0.656870
epoch 34; iter: 200; batch classifier loss: 0.666675; b

epoch 44; iter: 200; batch classifier loss: 0.630565
epoch 45; iter: 0; batch classifier loss: 0.673555
epoch 45; iter: 200; batch classifier loss: 0.654289
epoch 46; iter: 0; batch classifier loss: 0.606021
epoch 46; iter: 200; batch classifier loss: 0.615896
epoch 47; iter: 0; batch classifier loss: 0.576597
epoch 47; iter: 200; batch classifier loss: 0.575241
epoch 48; iter: 0; batch classifier loss: 0.602093
epoch 48; iter: 200; batch classifier loss: 0.601114
epoch 49; iter: 0; batch classifier loss: 0.575440
epoch 49; iter: 200; batch classifier loss: 0.567482
epoch 0; iter: 0; batch classifier loss: 0.673471; batch adversarial loss: 0.780975
epoch 1; iter: 0; batch classifier loss: 0.690468; batch adversarial loss: 0.829493
epoch 2; iter: 0; batch classifier loss: 0.667842; batch adversarial loss: 0.847133
epoch 3; iter: 0; batch classifier loss: 0.703228; batch adversarial loss: 0.870367
epoch 4; iter: 0; batch classifier loss: 0.765185; batch adversarial loss: 0.884526
epoch 5

In [12]:
privileged_groups = [{'age': 1}]
unprivileged_groups = [{'age': 0}]

print("Classification with German data set\n")
# df = pd.read_csv('dataset/compas-scores-two-years.csv')
dataset_orig = load_preproc_data_german()

matrix_accuracy_reweigh = {}
matrix_accuracy_nonreweigh = {}
matrix_fairness_reweigh = {}
matrix_fairness_nonreweigh = {}

runs = 10

for i in range(0, runs):
    print('run =', i+1)
    
    train, test = dataset_orig.split([0.7], shuffle=True)
    train_transformed = reweighing_data(train, unprivileged_groups, privileged_groups)
    
    # with reweighing
    accuracy_reweigh, fairness_metrics_reweigh = make_prediction(train_transformed, test, unprivileged_groups, privileged_groups)
    
    # Without reweighing
    accuracy_nonreweigh, fairness_metrics_nonreweigh = make_prediction(train, test, unprivileged_groups, privileged_groups)
    
    # store values for each run
    matrix_accuracy_reweigh[i] = accuracy_reweigh
    matrix_fairness_reweigh[i] = fairness_metrics_reweigh
    matrix_accuracy_nonreweigh[i] = accuracy_nonreweigh
    matrix_fairness_nonreweigh[i] = fairness_metrics_nonreweigh

print('\nprediction completed')

metrics_adversarial_reweigh = []
metrics_prejudice_reweigh = []
metrics_nondebiasing_reweigh = []
metrics_ensemble_reweigh = []
metrics_adversarial_nonreweigh = []
metrics_prejudice_nonreweigh = []
metrics_nondebiasing_nonreweigh = []
metrics_ensemble_nonreweigh = []

for i in range(0, runs):
    
    # with reweighing
    metrics_adversarial_reweigh.append(matrix_fairness_reweigh[i][0])
    metrics_prejudice_reweigh.append(matrix_fairness_reweigh[i][1])
    metrics_nondebiasing_reweigh.append(matrix_fairness_reweigh[i][2])
    metrics_ensemble_reweigh.append(matrix_fairness_reweigh[i][3])
    
    # without reweighing
    metrics_adversarial_nonreweigh.append(matrix_fairness_nonreweigh[i][0])
    metrics_prejudice_nonreweigh.append(matrix_fairness_nonreweigh[i][1])
    metrics_nondebiasing_nonreweigh.append(matrix_fairness_nonreweigh[i][2])
    metrics_ensemble_nonreweigh.append(matrix_fairness_nonreweigh[i][3])

print('compiled all metrics')


# create data frame for all metrics
columns = ['Adversarial Debiasing', 'Prejudice Remover', 'Nondebiasing', 'Ensemble']
accuracy_reweigh = np.array(list(matrix_accuracy_reweigh.values()))
accuracy_nonreweigh = np.array(list(matrix_accuracy_nonreweigh.values()))
compas_accuracy_reweigh = pd.DataFrame(accuracy_reweigh, columns=columns)
compas_accuracy_nonreweigh = pd.DataFrame(accuracy_nonreweigh, columns=columns)

# fairness metrics
columns = ['Mean Difference', 'Disparate Impact', 'Equal Opportunity Difference', 'Average Odds Difference', 'Theil Index']
compas_adversarial_reweigh = pd.DataFrame(metrics_adversarial_reweigh, columns=columns)
compas_prejudice_reweigh = pd.DataFrame(metrics_prejudice_reweigh, columns=columns)
compas_nondebiasing_reweigh = pd.DataFrame(metrics_nondebiasing_reweigh, columns=columns)
compas_ensemble_reweigh = pd.DataFrame(metrics_ensemble_reweigh, columns=columns)

compas_adversarial_nonreweigh = pd.DataFrame(metrics_adversarial_nonreweigh, columns=columns)
compas_prejudice_nonreweigh = pd.DataFrame(metrics_prejudice_nonreweigh, columns=columns)
compas_nondebiasing_nonreweigh = pd.DataFrame(metrics_nondebiasing_nonreweigh, columns=columns)
compas_ensemble_nonreweigh = pd.DataFrame(metrics_ensemble_nonreweigh, columns=columns)


# save to csv
compas_accuracy_reweigh.to_csv("results-11feb/race/german/reweighed/accuracy.csv", encoding='utf-8')
compas_adversarial_reweigh.to_csv("results-11feb/race/german/reweighed/adversarial.csv", encoding='utf-8')
compas_prejudice_reweigh.to_csv("results-11feb/race/german/reweighed/prejudice.csv", encoding='utf-8')
compas_nondebiasing_reweigh.to_csv("results-11feb/race/german/reweighed/neural_net.csv", encoding='utf-8')
compas_ensemble_reweigh.to_csv("results-11feb/race/german/reweighed/ensemble.csv", encoding='utf-8')

compas_accuracy_nonreweigh.to_csv("results-11feb/race/german/non-reweighed/accuracy.csv", encoding='utf-8')
compas_adversarial_nonreweigh.to_csv("results-11feb/race/german/non-reweighed/adversarial.csv", encoding='utf-8')
compas_prejudice_nonreweigh.to_csv("results-11feb/race/german/non-reweighed/prejudice.csv", encoding='utf-8')
compas_nondebiasing_nonreweigh.to_csv("results-11feb/race/german/non-reweighed/neural_net.csv", encoding='utf-8')
compas_ensemble_nonreweigh.to_csv("results-11feb/race/german/non-reweighed/ensemble.csv", encoding='utf-8')

print('all files saved to csv')

Classification with German data set

run = 1
epoch 0; iter: 0; batch classifier loss: 0.674728; batch adversarial loss: 0.507687
epoch 1; iter: 0; batch classifier loss: 0.676047; batch adversarial loss: 0.535870
epoch 2; iter: 0; batch classifier loss: 0.710204; batch adversarial loss: 0.658102
epoch 3; iter: 0; batch classifier loss: 1.022655; batch adversarial loss: 0.730948
epoch 4; iter: 0; batch classifier loss: 1.252913; batch adversarial loss: 0.649106
epoch 5; iter: 0; batch classifier loss: 1.316567; batch adversarial loss: 0.664673
epoch 6; iter: 0; batch classifier loss: 1.489223; batch adversarial loss: 0.670780
epoch 7; iter: 0; batch classifier loss: 1.495940; batch adversarial loss: 0.649649
epoch 8; iter: 0; batch classifier loss: 1.499763; batch adversarial loss: 0.613185
epoch 9; iter: 0; batch classifier loss: 1.470350; batch adversarial loss: 0.575080
epoch 10; iter: 0; batch classifier loss: 1.445695; batch adversarial loss: 0.532479
epoch 11; iter: 0; batch class

epoch 27; iter: 0; batch classifier loss: 0.522979; batch adversarial loss: 0.656417
epoch 28; iter: 0; batch classifier loss: 0.554475; batch adversarial loss: 0.653743
epoch 29; iter: 0; batch classifier loss: 0.518278; batch adversarial loss: 0.660175
epoch 30; iter: 0; batch classifier loss: 0.586933; batch adversarial loss: 0.641512
epoch 31; iter: 0; batch classifier loss: 0.580953; batch adversarial loss: 0.629487
epoch 32; iter: 0; batch classifier loss: 0.663458; batch adversarial loss: 0.626686
epoch 33; iter: 0; batch classifier loss: 0.597421; batch adversarial loss: 0.638406
epoch 34; iter: 0; batch classifier loss: 0.491358; batch adversarial loss: 0.618261
epoch 35; iter: 0; batch classifier loss: 0.572421; batch adversarial loss: 0.639410
epoch 36; iter: 0; batch classifier loss: 0.578668; batch adversarial loss: 0.628781
epoch 37; iter: 0; batch classifier loss: 0.525672; batch adversarial loss: 0.622967
epoch 38; iter: 0; batch classifier loss: 0.571856; batch adversa

epoch 44; iter: 0; batch classifier loss: 0.602327; batch adversarial loss: 0.418692
epoch 45; iter: 0; batch classifier loss: 0.617552; batch adversarial loss: 0.416115
epoch 46; iter: 0; batch classifier loss: 0.571668; batch adversarial loss: 0.388498
epoch 47; iter: 0; batch classifier loss: 0.548667; batch adversarial loss: 0.414087
epoch 48; iter: 0; batch classifier loss: 0.604904; batch adversarial loss: 0.403004
epoch 49; iter: 0; batch classifier loss: 0.539955; batch adversarial loss: 0.353197
epoch 0; iter: 0; batch classifier loss: 0.688293
epoch 1; iter: 0; batch classifier loss: 0.621244
epoch 2; iter: 0; batch classifier loss: 0.601939
epoch 3; iter: 0; batch classifier loss: 0.614821
epoch 4; iter: 0; batch classifier loss: 0.518311
epoch 5; iter: 0; batch classifier loss: 0.602846
epoch 6; iter: 0; batch classifier loss: 0.561570
epoch 7; iter: 0; batch classifier loss: 0.578488
epoch 8; iter: 0; batch classifier loss: 0.560114
epoch 9; iter: 0; batch classifier loss:

epoch 32; iter: 0; batch classifier loss: 0.572948
epoch 33; iter: 0; batch classifier loss: 0.554013
epoch 34; iter: 0; batch classifier loss: 0.533113
epoch 35; iter: 0; batch classifier loss: 0.591745
epoch 36; iter: 0; batch classifier loss: 0.542216
epoch 37; iter: 0; batch classifier loss: 0.546634
epoch 38; iter: 0; batch classifier loss: 0.639741
epoch 39; iter: 0; batch classifier loss: 0.566085
epoch 40; iter: 0; batch classifier loss: 0.585894
epoch 41; iter: 0; batch classifier loss: 0.607124
epoch 42; iter: 0; batch classifier loss: 0.579273
epoch 43; iter: 0; batch classifier loss: 0.567551
epoch 44; iter: 0; batch classifier loss: 0.612624
epoch 45; iter: 0; batch classifier loss: 0.614969
epoch 46; iter: 0; batch classifier loss: 0.633535
epoch 47; iter: 0; batch classifier loss: 0.610719
epoch 48; iter: 0; batch classifier loss: 0.553526
epoch 49; iter: 0; batch classifier loss: 0.623612
run = 3
epoch 0; iter: 0; batch classifier loss: 0.711024; batch adversarial loss:

/home/dani/anaconda3/envs/ml-19jan/lib/python3.5/site-packages/aif360/metrics/dataset_metric.py:94: RuntimeWarning: invalid value encountered in double_scalars
  return metric_fun(privileged=False) / metric_fun(privileged=True)


...

epoch 0; iter: 0; batch classifier loss: 0.708093
epoch 1; iter: 0; batch classifier loss: 0.642239
epoch 2; iter: 0; batch classifier loss: 0.626431
epoch 3; iter: 0; batch classifier loss: 0.557786
epoch 4; iter: 0; batch classifier loss: 0.608361
epoch 5; iter: 0; batch classifier loss: 0.597280
epoch 6; iter: 0; batch classifier loss: 0.646842
epoch 7; iter: 0; batch classifier loss: 0.561929
epoch 8; iter: 0; batch classifier loss: 0.602368
epoch 9; iter: 0; batch classifier loss: 0.575522
epoch 10; iter: 0; batch classifier loss: 0.564809
epoch 11; iter: 0; batch classifier loss: 0.515048
epoch 12; iter: 0; batch classifier loss: 0.627529
epoch 13; iter: 0; batch classifier loss: 0.602081
epoch 14; iter: 0; batch classifier loss: 0.644184
epoch 15; iter: 0; batch classifier loss: 0.612060
epoch 16; iter: 0; batch classifier loss: 0.590340
epoch 17; iter: 0; batch classifier loss: 0.638449
epoch 18; iter: 0; batch classifier loss: 0.606916
epoch 19; iter: 0; batch classifier loss:

epoch 28; iter: 0; batch classifier loss: 0.556675
epoch 29; iter: 0; batch classifier loss: 0.477600
epoch 30; iter: 0; batch classifier loss: 0.604063
epoch 31; iter: 0; batch classifier loss: 0.449450
epoch 32; iter: 0; batch classifier loss: 0.506316
epoch 33; iter: 0; batch classifier loss: 0.473018
epoch 34; iter: 0; batch classifier loss: 0.488924
epoch 35; iter: 0; batch classifier loss: 0.528444
epoch 36; iter: 0; batch classifier loss: 0.606351
epoch 37; iter: 0; batch classifier loss: 0.505449
epoch 38; iter: 0; batch classifier loss: 0.550330
epoch 39; iter: 0; batch classifier loss: 0.494101
epoch 40; iter: 0; batch classifier loss: 0.518417
epoch 41; iter: 0; batch classifier loss: 0.567425
epoch 42; iter: 0; batch classifier loss: 0.499328
epoch 43; iter: 0; batch classifier loss: 0.539350
epoch 44; iter: 0; batch classifier loss: 0.573059
epoch 45; iter: 0; batch classifier loss: 0.557557
epoch 46; iter: 0; batch classifier loss: 0.481445
epoch 47; iter: 0; batch classi

/home/dani/anaconda3/envs/ml-19jan/lib/python3.5/site-packages/aif360/metrics/dataset_metric.py:94: RuntimeWarning: divide by zero encountered in double_scalars
  return metric_fun(privileged=False) / metric_fun(privileged=True)


epoch 0; iter: 0; batch classifier loss: 0.702313
epoch 1; iter: 0; batch classifier loss: 0.653784
epoch 2; iter: 0; batch classifier loss: 0.624428
epoch 3; iter: 0; batch classifier loss: 0.530635
epoch 4; iter: 0; batch classifier loss: 0.502238
epoch 5; iter: 0; batch classifier loss: 0.631842
epoch 6; iter: 0; batch classifier loss: 0.504143
epoch 7; iter: 0; batch classifier loss: 0.691968
epoch 8; iter: 0; batch classifier loss: 0.530739
epoch 9; iter: 0; batch classifier loss: 0.539838
epoch 10; iter: 0; batch classifier loss: 0.535095
epoch 11; iter: 0; batch classifier loss: 0.539878
epoch 12; iter: 0; batch classifier loss: 0.581040
epoch 13; iter: 0; batch classifier loss: 0.539105
epoch 14; iter: 0; batch classifier loss: 0.548719
epoch 15; iter: 0; batch classifier loss: 0.510625
epoch 16; iter: 0; batch classifier loss: 0.528183
epoch 17; iter: 0; batch classifier loss: 0.458212
epoch 18; iter: 0; batch classifier loss: 0.549639
epoch 19; iter: 0; batch classifier loss:

epoch 28; iter: 0; batch classifier loss: 0.577733
epoch 29; iter: 0; batch classifier loss: 0.507871
epoch 30; iter: 0; batch classifier loss: 0.575643
epoch 31; iter: 0; batch classifier loss: 0.512980
epoch 32; iter: 0; batch classifier loss: 0.608648
epoch 33; iter: 0; batch classifier loss: 0.531883
epoch 34; iter: 0; batch classifier loss: 0.478486
epoch 35; iter: 0; batch classifier loss: 0.565017
epoch 36; iter: 0; batch classifier loss: 0.539549
epoch 37; iter: 0; batch classifier loss: 0.511149
epoch 38; iter: 0; batch classifier loss: 0.490012
epoch 39; iter: 0; batch classifier loss: 0.483139
epoch 40; iter: 0; batch classifier loss: 0.500665
epoch 41; iter: 0; batch classifier loss: 0.615146
epoch 42; iter: 0; batch classifier loss: 0.501526
epoch 43; iter: 0; batch classifier loss: 0.486841
epoch 44; iter: 0; batch classifier loss: 0.561623
epoch 45; iter: 0; batch classifier loss: 0.568142
epoch 46; iter: 0; batch classifier loss: 0.511183
epoch 47; iter: 0; batch classi

In [13]:
privileged_groups = [{'race': 1}]
unprivileged_groups = [{'race': 0}]

print("Classification with Adult data set\n")
dataset_orig = load_preproc_data_adult()

matrix_accuracy_reweigh = {}
matrix_accuracy_nonreweigh = {}
matrix_fairness_reweigh = {}
matrix_fairness_nonreweigh = {}

runs = 10

for i in range(0, runs):
    print('run =', i+1)
    
    train, test = dataset_orig.split([0.7], shuffle=True)
    train_transformed = reweighing_data(train, unprivileged_groups, privileged_groups)
    
    # with reweighing
    accuracy_reweigh, fairness_metrics_reweigh = make_prediction(train_transformed, test, unprivileged_groups, privileged_groups)
    
    # Without reweighing
    accuracy_nonreweigh, fairness_metrics_nonreweigh = make_prediction(train, test, unprivileged_groups, privileged_groups)
    
    # store values for each run
    matrix_accuracy_reweigh[i] = accuracy_reweigh
    matrix_fairness_reweigh[i] = fairness_metrics_reweigh
    matrix_accuracy_nonreweigh[i] = accuracy_nonreweigh
    matrix_fairness_nonreweigh[i] = fairness_metrics_nonreweigh

print('\nprediction completed')

metrics_adversarial_reweigh = []
metrics_prejudice_reweigh = []
metrics_nondebiasing_reweigh = []
metrics_ensemble_reweigh = []
metrics_adversarial_nonreweigh = []
metrics_prejudice_nonreweigh = []
metrics_nondebiasing_nonreweigh = []
metrics_ensemble_nonreweigh = []

for i in range(0, runs):
    
    # with reweighing
    metrics_adversarial_reweigh.append(matrix_fairness_reweigh[i][0])
    metrics_prejudice_reweigh.append(matrix_fairness_reweigh[i][1])
    metrics_nondebiasing_reweigh.append(matrix_fairness_reweigh[i][2])
    metrics_ensemble_reweigh.append(matrix_fairness_reweigh[i][3])
    
    # without reweighing
    metrics_adversarial_nonreweigh.append(matrix_fairness_nonreweigh[i][0])
    metrics_prejudice_nonreweigh.append(matrix_fairness_nonreweigh[i][1])
    metrics_nondebiasing_nonreweigh.append(matrix_fairness_nonreweigh[i][2])
    metrics_ensemble_nonreweigh.append(matrix_fairness_nonreweigh[i][3])

print('compiled all metrics')


# create data frame for all metrics
columns = ['Adversarial Debiasing', 'Prejudice Remover', 'Nondebiasing', 'Ensemble']
accuracy_reweigh = np.array(list(matrix_accuracy_reweigh.values()))
accuracy_nonreweigh = np.array(list(matrix_accuracy_nonreweigh.values()))
adult_accuracy_reweigh = pd.DataFrame(accuracy_reweigh, columns=columns)
adult_accuracy_nonreweigh = pd.DataFrame(accuracy_nonreweigh, columns=columns)

# fairness metrics
columns = ['Mean Difference', 'Disparate Impact', 'Equal Opportunity Difference', 'Average Odds Difference', 'Theil Index']
adult_adversarial_reweigh = pd.DataFrame(metrics_adversarial_reweigh, columns=columns)
adult_prejudice_reweigh = pd.DataFrame(metrics_prejudice_reweigh, columns=columns)
adult_nondebiasing_reweigh = pd.DataFrame(metrics_nondebiasing_reweigh, columns=columns)
adult_ensemble_reweigh = pd.DataFrame(metrics_ensemble_reweigh, columns=columns)

adult_adversarial_nonreweigh = pd.DataFrame(metrics_adversarial_nonreweigh, columns=columns)
adult_prejudice_nonreweigh = pd.DataFrame(metrics_prejudice_nonreweigh, columns=columns)
adult_nondebiasing_nonreweigh = pd.DataFrame(metrics_nondebiasing_nonreweigh, columns=columns)
adult_ensemble_nonreweigh = pd.DataFrame(metrics_ensemble_nonreweigh, columns=columns)


# save to csv
adult_accuracy_reweigh.to_csv("results-11feb/race/adult/reweighed/accuracy.csv", encoding='utf-8')
adult_adversarial_reweigh.to_csv("results-11feb/race/adult/reweighed/adversarial.csv", encoding='utf-8')
adult_prejudice_reweigh.to_csv("results-11feb/race/adult/reweighed/prejudice.csv", encoding='utf-8')
adult_nondebiasing_reweigh.to_csv("results-11feb/race/adult/reweighed/neural_net.csv", encoding='utf-8')
adult_ensemble_reweigh.to_csv("results-11feb/race/adult/reweighed/ensemble.csv", encoding='utf-8')

adult_accuracy_nonreweigh.to_csv("results-11feb/race/adult/non-reweighed/accuracy.csv", encoding='utf-8')
adult_adversarial_nonreweigh.to_csv("results-11feb/race/adult/non-reweighed/adversarial.csv", encoding='utf-8')
adult_prejudice_nonreweigh.to_csv("results-11feb/race/adult/non-reweighed/prejudice.csv", encoding='utf-8')
adult_nondebiasing_nonreweigh.to_csv("results-11feb/race/adult/non-reweighed/neural_net.csv", encoding='utf-8')
adult_ensemble_nonreweigh.to_csv("results-11feb/race/adult/non-reweighed/ensemble.csv", encoding='utf-8')

print('all files saved to csv')

Classification with Adult data set

run = 1
epoch 0; iter: 0; batch classifier loss: 0.754099; batch adversarial loss: 0.745721
epoch 0; iter: 200; batch classifier loss: 0.490396; batch adversarial loss: 0.640205
epoch 0; iter: 400; batch classifier loss: 0.418454; batch adversarial loss: 0.584018
epoch 0; iter: 600; batch classifier loss: 0.382492; batch adversarial loss: 0.523020
epoch 0; iter: 800; batch classifier loss: 0.437498; batch adversarial loss: 0.467506
epoch 0; iter: 1000; batch classifier loss: 0.460201; batch adversarial loss: 0.498694
epoch 0; iter: 1200; batch classifier loss: 0.390761; batch adversarial loss: 0.395536
epoch 0; iter: 1400; batch classifier loss: 0.422271; batch adversarial loss: 0.539811
epoch 0; iter: 1600; batch classifier loss: 0.454482; batch adversarial loss: 0.475391
epoch 0; iter: 1800; batch classifier loss: 0.388615; batch adversarial loss: 0.430075
epoch 0; iter: 2000; batch classifier loss: 0.501411; batch adversarial loss: 0.518137
epoch 

epoch 6; iter: 2200; batch classifier loss: 0.480280; batch adversarial loss: 0.489400
epoch 6; iter: 2400; batch classifier loss: 0.440526; batch adversarial loss: 0.431703
epoch 6; iter: 2600; batch classifier loss: 0.383873; batch adversarial loss: 0.444845
epoch 7; iter: 0; batch classifier loss: 0.329551; batch adversarial loss: 0.407541
epoch 7; iter: 200; batch classifier loss: 0.361848; batch adversarial loss: 0.378791
epoch 7; iter: 400; batch classifier loss: 0.409997; batch adversarial loss: 0.420720
epoch 7; iter: 600; batch classifier loss: 0.382279; batch adversarial loss: 0.462574
epoch 7; iter: 800; batch classifier loss: 0.331458; batch adversarial loss: 0.408880
epoch 7; iter: 1000; batch classifier loss: 0.419890; batch adversarial loss: 0.418779
epoch 7; iter: 1200; batch classifier loss: 0.416843; batch adversarial loss: 0.365991
epoch 7; iter: 1400; batch classifier loss: 0.335010; batch adversarial loss: 0.365995
epoch 7; iter: 1600; batch classifier loss: 0.4210

epoch 13; iter: 1600; batch classifier loss: 0.391217; batch adversarial loss: 0.420082
epoch 13; iter: 1800; batch classifier loss: 0.401406; batch adversarial loss: 0.407822
epoch 13; iter: 2000; batch classifier loss: 0.540501; batch adversarial loss: 0.363910
epoch 13; iter: 2200; batch classifier loss: 0.391011; batch adversarial loss: 0.379258
epoch 13; iter: 2400; batch classifier loss: 0.422675; batch adversarial loss: 0.365995
epoch 13; iter: 2600; batch classifier loss: 0.416614; batch adversarial loss: 0.446188
epoch 14; iter: 0; batch classifier loss: 0.378994; batch adversarial loss: 0.351728
epoch 14; iter: 200; batch classifier loss: 0.298794; batch adversarial loss: 0.433250
epoch 14; iter: 400; batch classifier loss: 0.418559; batch adversarial loss: 0.353160
epoch 14; iter: 600; batch classifier loss: 0.475833; batch adversarial loss: 0.402977
epoch 14; iter: 800; batch classifier loss: 0.450172; batch adversarial loss: 0.337027
epoch 14; iter: 1000; batch classifier 

epoch 20; iter: 800; batch classifier loss: 0.403659; batch adversarial loss: 0.490536
epoch 20; iter: 1000; batch classifier loss: 0.351765; batch adversarial loss: 0.392317
epoch 20; iter: 1200; batch classifier loss: 0.413582; batch adversarial loss: 0.407055
epoch 20; iter: 1400; batch classifier loss: 0.377743; batch adversarial loss: 0.364352
epoch 20; iter: 1600; batch classifier loss: 0.453260; batch adversarial loss: 0.379150
epoch 20; iter: 1800; batch classifier loss: 0.482487; batch adversarial loss: 0.405900
epoch 20; iter: 2000; batch classifier loss: 0.305830; batch adversarial loss: 0.430752
epoch 20; iter: 2200; batch classifier loss: 0.365335; batch adversarial loss: 0.419711
epoch 20; iter: 2400; batch classifier loss: 0.320365; batch adversarial loss: 0.423927
epoch 20; iter: 2600; batch classifier loss: 0.459641; batch adversarial loss: 0.490478
epoch 21; iter: 0; batch classifier loss: 0.395610; batch adversarial loss: 0.379204
epoch 21; iter: 200; batch classifie

epoch 27; iter: 200; batch classifier loss: 0.419108; batch adversarial loss: 0.392055
epoch 27; iter: 400; batch classifier loss: 0.466198; batch adversarial loss: 0.407188
epoch 27; iter: 600; batch classifier loss: 0.490239; batch adversarial loss: 0.325302
epoch 27; iter: 800; batch classifier loss: 0.419466; batch adversarial loss: 0.351012
epoch 27; iter: 1000; batch classifier loss: 0.411487; batch adversarial loss: 0.364654
epoch 27; iter: 1200; batch classifier loss: 0.343702; batch adversarial loss: 0.366087
epoch 27; iter: 1400; batch classifier loss: 0.413363; batch adversarial loss: 0.392481
epoch 27; iter: 1600; batch classifier loss: 0.460624; batch adversarial loss: 0.379024
epoch 27; iter: 1800; batch classifier loss: 0.390951; batch adversarial loss: 0.434569
epoch 27; iter: 2000; batch classifier loss: 0.383666; batch adversarial loss: 0.405670
epoch 27; iter: 2200; batch classifier loss: 0.363008; batch adversarial loss: 0.405675
epoch 27; iter: 2400; batch classifi

...

epoch 21; iter: 2200; batch classifier loss: 0.384206
epoch 21; iter: 2400; batch classifier loss: 0.464515
epoch 21; iter: 2600; batch classifier loss: 0.323959
epoch 22; iter: 0; batch classifier loss: 0.376280
epoch 22; iter: 200; batch classifier loss: 0.422313
epoch 22; iter: 400; batch classifier loss: 0.460690
epoch 22; iter: 600; batch classifier loss: 0.348093
epoch 22; iter: 800; batch classifier loss: 0.403570
epoch 22; iter: 1000; batch classifier loss: 0.412800
epoch 22; iter: 1200; batch classifier loss: 0.412587
epoch 22; iter: 1400; batch classifier loss: 0.368787
epoch 22; iter: 1600; batch classifier loss: 0.332754
epoch 22; iter: 1800; batch classifier loss: 0.411166
epoch 22; iter: 2000; batch classifier loss: 0.531528
epoch 22; iter: 2200; batch classifier loss: 0.481967
epoch 22; iter: 2400; batch classifier loss: 0.467074
epoch 22; iter: 2600; batch classifier loss: 0.383130
epoch 23; iter: 0; batch classifier loss: 0.320191
epoch 23; iter: 200; batch classifier 

epoch 32; iter: 2200; batch classifier loss: 0.434445
epoch 32; iter: 2400; batch classifier loss: 0.414074
epoch 32; iter: 2600; batch classifier loss: 0.293256
epoch 33; iter: 0; batch classifier loss: 0.458931
epoch 33; iter: 200; batch classifier loss: 0.420616
epoch 33; iter: 400; batch classifier loss: 0.317273
epoch 33; iter: 600; batch classifier loss: 0.403576
epoch 33; iter: 800; batch classifier loss: 0.514409
epoch 33; iter: 1000; batch classifier loss: 0.512089
epoch 33; iter: 1200; batch classifier loss: 0.448947
epoch 33; iter: 1400; batch classifier loss: 0.406845
epoch 33; iter: 1600; batch classifier loss: 0.371962
epoch 33; iter: 1800; batch classifier loss: 0.330346
epoch 33; iter: 2000; batch classifier loss: 0.409295
epoch 33; iter: 2200; batch classifier loss: 0.366557
epoch 33; iter: 2400; batch classifier loss: 0.438069
epoch 33; iter: 2600; batch classifier loss: 0.380374
epoch 34; iter: 0; batch classifier loss: 0.445388
epoch 34; iter: 200; batch classifier 

epoch 43; iter: 2200; batch classifier loss: 0.476779
epoch 43; iter: 2400; batch classifier loss: 0.477373
epoch 43; iter: 2600; batch classifier loss: 0.488902
epoch 44; iter: 0; batch classifier loss: 0.289174
epoch 44; iter: 200; batch classifier loss: 0.439974
epoch 44; iter: 400; batch classifier loss: 0.391435
epoch 44; iter: 600; batch classifier loss: 0.337072
epoch 44; iter: 800; batch classifier loss: 0.411448
epoch 44; iter: 1000; batch classifier loss: 0.425814
epoch 44; iter: 1200; batch classifier loss: 0.375711
epoch 44; iter: 1400; batch classifier loss: 0.490960
epoch 44; iter: 1600; batch classifier loss: 0.377234
epoch 44; iter: 1800; batch classifier loss: 0.377988
epoch 44; iter: 2000; batch classifier loss: 0.374928
epoch 44; iter: 2200; batch classifier loss: 0.364781
epoch 44; iter: 2400; batch classifier loss: 0.452705
epoch 44; iter: 2600; batch classifier loss: 0.514269
epoch 45; iter: 0; batch classifier loss: 0.392461
epoch 45; iter: 200; batch classifier 

epoch 21; iter: 0; batch classifier loss: 0.411605; batch adversarial loss: 0.376724
epoch 21; iter: 200; batch classifier loss: 0.408953; batch adversarial loss: 0.436764
epoch 22; iter: 0; batch classifier loss: 0.476702; batch adversarial loss: 0.420509
epoch 22; iter: 200; batch classifier loss: 0.378084; batch adversarial loss: 0.388220
epoch 23; iter: 0; batch classifier loss: 0.400600; batch adversarial loss: 0.320062
epoch 23; iter: 200; batch classifier loss: 0.371322; batch adversarial loss: 0.436332
epoch 24; iter: 0; batch classifier loss: 0.418785; batch adversarial loss: 0.445719
epoch 24; iter: 200; batch classifier loss: 0.459529; batch adversarial loss: 0.411823
epoch 25; iter: 0; batch classifier loss: 0.400217; batch adversarial loss: 0.347935
epoch 25; iter: 200; batch classifier loss: 0.352912; batch adversarial loss: 0.343624
epoch 26; iter: 0; batch classifier loss: 0.359474; batch adversarial loss: 0.487309
epoch 26; iter: 200; batch classifier loss: 0.357444; b

epoch 31; iter: 200; batch classifier loss: 0.353875
epoch 32; iter: 0; batch classifier loss: 0.445848
epoch 32; iter: 200; batch classifier loss: 0.405246
epoch 33; iter: 0; batch classifier loss: 0.472004
epoch 33; iter: 200; batch classifier loss: 0.446375
epoch 34; iter: 0; batch classifier loss: 0.415029
epoch 34; iter: 200; batch classifier loss: 0.390512
epoch 35; iter: 0; batch classifier loss: 0.398497
epoch 35; iter: 200; batch classifier loss: 0.406129
epoch 36; iter: 0; batch classifier loss: 0.394259
epoch 36; iter: 200; batch classifier loss: 0.367196
epoch 37; iter: 0; batch classifier loss: 0.457523
epoch 37; iter: 200; batch classifier loss: 0.401122
epoch 38; iter: 0; batch classifier loss: 0.412706
epoch 38; iter: 200; batch classifier loss: 0.385262
epoch 39; iter: 0; batch classifier loss: 0.478746
epoch 39; iter: 200; batch classifier loss: 0.414013
epoch 40; iter: 0; batch classifier loss: 0.385285
epoch 40; iter: 200; batch classifier loss: 0.429646
epoch 41; i